---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [3]:
import pandas as pd
import numpy as np 
from re import search

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    
    Town = pd.read_csv("university_towns.txt",sep='\n', header=None, names=['RegionName'])
    # Home = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    
    Town["State"] = Town['RegionName'][Town['RegionName'].str.contains("edit")].str.replace(r'\[edit]', '') # find the state name using str.contains
    Town["State"] = Town["State"].fillna(method = 'ffill') # forward fill the NA values by the state name 
    Town["RegionName"] = Town["RegionName"].str.split("(", n= 1, expand = True)[0].str.split('[', expand=True)[0].str.rstrip() # split the column into two columns according to "("
    Town2 = Town[["State","RegionName"]]
    Town2 = Town2[Town2["State"] != Town2["RegionName"]] # remove the state from rows 
    

    states_rev = {v: k for k, v in states.items()} # reverse the index and values in dictionary 
    Town2["Abb"] = [states_rev[state] for state in Town2["State"]] # link Abb to the corresponding RegionName


    return Town2
        

get_list_of_university_towns()

#x[x['State'] == 'Massachusetts']

,State,RegionName,Abb
1,Alabama,Auburn,AL
2,Alabama,Florence,AL
3,Alabama,Jacksonville,AL
4,Alabama,Livingston,AL
5,Alabama,Montevallo,AL
...,...,...,...
561,Wisconsin,River Falls,WI
562,Wisconsin,Stevens Point,WI
563,Wisconsin,Waukesha,WI
564,Wisconsin,Whitewater,WI


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 5) # drop 4 rows 
    GDP = GDP.drop(GDP.columns[[0,1,2,3,5,7]], axis = 1) # drop columns 
    GDP.columns = ["Year Quarter","GDP in billions of chained 2009 dollars"]
    
    # GDP data from the first quarter of 2000 onward
    GDP = GDP[GDP["Year Quarter"] >= "2000q1"]
    
    # get difference between consecutive rows 
    GDP["diff"] = GDP["GDP in billions of chained 2009 dollars"].diff()
    
    # assign 1 to consecutive positive diff,  -1 to consecutive negative diff, and 0 to all other cases
    concession = {}
    for i in range(len(GDP["diff"])-1):
        if ((GDP["diff"].iloc[i] <0) & (GDP["diff"].iloc[i+1] <0)):
            concession[i] = -1
        elif ((GDP["diff"].iloc[i-1] >0) & (GDP["diff"].iloc[i] >0)):
            concession[i] = 1
        else:
            concession[i] = 0
    concession[len(GDP["diff"])] = "NA"
            
    GDP["concession"] = concession.values()
    
    # get the first concession using iloc[0]
    start = GDP[GDP["concession"] == -1].iloc[0][0]
    

    
    
    return start
    

   

get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 5) # drop 4 rows 
    GDP = GDP.drop(GDP.columns[[0,1,2,3,5,7]], axis = 1) # drop columns 
    GDP.columns = ["Year Quarter","GDP in billions of chained 2009 dollars"]
    
    # GDP data from the first quarter of 2000 onward
    GDP = GDP[GDP["Year Quarter"] >= "2000q1"]
    
    # get difference between consecutive rows 
    GDP["diff"] = GDP["GDP in billions of chained 2009 dollars"].diff()
    
    # assign 1 to consecutive positive diff,  -1 to consecutive negative diff, and 0 to all other cases
    concession = {}
    for i in range(len(GDP["diff"])-1):
        if ((GDP["diff"].iloc[i] <0) & (GDP["diff"].iloc[i+1] <0)):
            concession[i] = -1
        elif ((GDP["diff"].iloc[i-1] >0) & (GDP["diff"].iloc[i] >0)):
            concession[i] = 1
        else:
            concession[i] = 0
    concession[len(GDP["diff"])] = "NA"
            
    GDP["concession"] = concession.values()
    
    # get the first decline using iloc[0]
    start = GDP[GDP["concession"] == -1].iloc[0][0]
    
    # get the first growth after the decline 
    end = GDP[(GDP["concession"] == 1) & (GDP["Year Quarter"] > start)].iloc[0][0]
    
    return end
    

get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 5) # drop 4 rows 
    GDP = GDP.drop(GDP.columns[[0,1,2,3,5,7]], axis = 1) # drop columns 
    GDP.columns = ["Year Quarter","GDP in billions of chained 2009 dollars"]
    
    # GDP data from the first quarter of 2000 onward
    GDP = GDP[GDP["Year Quarter"] >= "2000q1"]
    
    # get difference between consecutive rows 
    GDP["diff"] = GDP["GDP in billions of chained 2009 dollars"].diff()
    
    # assign 1 to consecutive positive diff,  -1 to consecutive negative diff, and 0 to all other cases
    concession = {}
    for i in range(len(GDP["diff"])-1):
        if ((GDP["diff"].iloc[i] <0) & (GDP["diff"].iloc[i+1] <0)):
            concession[i] = -1
        elif ((GDP["diff"].iloc[i-1] >0) & (GDP["diff"].iloc[i] >0)):
            concession[i] = 1
        else:
            concession[i] = 0
    concession[len(GDP["diff"])] = "NA"
            
    GDP["concession"] = concession.values()
    
    # get the first decline using iloc[0]
    start = GDP[GDP["concession"] == -1].iloc[0][0]
    
    # get the first growth after the decline 
    end = GDP[(GDP["concession"] == 1) & (GDP["Year Quarter"] > start)].iloc[0][0]
    
    # get the lowest GDP time
    bottom = GDP[(GDP["Year Quarter"] >= start) & (GDP["Year Quarter"] <= end)]["GDP in billions of chained 2009 dollars"].idxmin()
    GDP_bottom = GDP["Year Quarter"][bottom]
    
    return GDP_bottom
    
    
    

get_recession_bottom()

'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import pandas as pd
    Home = pd.read_csv("City_Zhvi_AllHomes.csv")
    
    # convert the column from strings to period 
    Home.columns = Home.columns[:6].tolist() + pd.to_datetime(Home.columns[6:]).to_period('M').tolist() 
    
    # delete the columns beyond 2000q1 through 2016q3
    new_column = Home.columns[6:][(Home.columns[6:] < pd.Period("01/2000")) | (Home.columns[6:] > pd.Period("09/2016"))]
    Home = Home.drop(columns=new_column)
    Home = Home.drop(columns = ["RegionID","Metro", "CountyName","SizeRank"])
    
    # get dataframe Home1
    Home1 = Home[["RegionName","State"]]
    
    # get dataframe Home2 which is the mean of housing data in each quarter
    Home.columns = Home.columns[:2].tolist()+ Home.columns[2:].to_series().astype(str).tolist() # convert period type to string
    Home2 = (Home.iloc[:,2:].groupby(pd.PeriodIndex(Home.columns[2:].astype("period[Q]"), freq='Q'), axis=1)\
    .mean()\
    .rename(columns=lambda c: str(c).lower())) # calculate mean 
    
    # combine two dataframes: Home 1 and Home 2 
    Home = pd.concat([Home1, Home2], axis=1)
    
    # change State Abb to State
    Home["State"] = [states[abb] for abb in Home["State"]]
    
    # set multi-index
    Home_final = Home.set_index(["RegionName","State"])


    return Home_final
    #return pd.PeriodIndex(Home.columns[2:], freq='Q')

convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
RegionName,State,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
Los Angeles,California,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Chicago,Illinois,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Philadelphia,Pennsylvania,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Phoenix,Arizona,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Town of Wrightstown,Wisconsin,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
Urbana,New York,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
New Denmark,Wisconsin,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [9]:
def run_ttest():
    import pandas as pd
    from scipy import stats
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    start = get_recession_start()
    startb1 = str(pd.Period(start) - 1).lower() # get one quarter before recession starts
    GDP_bottom = get_recession_bottom()
    Home_final = convert_housing_data_to_quarters()
    
    # calculate the ratio = quarter_before_recession/recession_bottom
    ratio = (Home_final[startb1]/Home_final[GDP_bottom] ).to_frame()
    ratio.columns = ["Ratio"]
    university_town_list = get_list_of_university_towns().set_index(["RegionName","State"])
    
    # find university town:
    Utown =  pd.merge(university_town_list, ratio, how = 'inner', left_index = True, right_index = True)
    Utown_price = Utown.Ratio
    
    # find not university town: 
    NotUtown = ratio[(~ratio.Ratio.isin(Utown.Ratio))]
    NotUtown_price = NotUtown.Ratio
    
    
    # run t-test:
    p = stats.ttest_ind(Utown_price.dropna(), NotUtown_price.dropna()).pvalue
    
    different = True if p<0.01 else False
    
    better = "university town" if Utown_price.mean()< NotUtown_price.mean() else "non-university town"
    
    
   

    
    
    
    return (p, different, better)
    
    
    
    
    
run_ttest()

(0.0027022679237793993, True, 'university town')